## this file will load snapshots and store details of the tracked star cluster thoroughout the snapshots into individual files available at ./data


### Given that I don't have all snapshots downloaded, I ran into issues of tracking clusters at unevenly spaced snapshots (code was going from snap 1 to snap 1+1. If snap 2 did not exsit, it didn't know how to jumpt to the next available snapshot. This code fixes this issue).


In [1]:
from fof_analysis import fof

above cell add the for loop and store into pkl


In [2]:
import gizmo_analysis as gizmo
import utilities as ut
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sl_utilities import distinct_colours as dc
from sl_utilities import distance_functions
import os
import pickle

In [13]:
#read in the simulation
simname = 'm12i_r7100'
simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/output/'


In [7]:
# creating a loop of snapshots (to track clusters over long periods of time)

# snapshot_start=446
# snapshot_end=600 

#snaps = [412,446,486,534,590,591,592,593,594,595,596,597,598,599,600]

#Loading the sample cluster to be tracked and sorting its id and id_child
# snapnumber=590 #Remember to change it if you  are changing the star cluster you are tracking in given snapshot

path="./"
if not os.path.exists(path):
  os.makedirs(path)


temporary_name ="fire2_clusters_m12i_res7100_snapshot_600_5_0_age_6_0.pkl"
# temporary_name = simname+'.'+str(snaps[0])+".minage"+ str('%.2f' %min_age_array_Gyr[k])+"_"+"maxage"+"_"+str('%.2f' %max_age_array_Gyr[k])+"_b"+str(b_parsec)+"_nmin"+str(ncut_min)+".pkl"
with open(path+temporary_name, "rb") as fp:
    cluster_dic = pickle.load(fp)


In [8]:
def matchids(id_current,id_child_current,id_next,id_child_next,id_generation_next): #this function returns the index of the ids in the next snapshot that match with the current
    ind=np.array(0)
    for i in range(len(id_current)):
        match=np.where((id_next==id_current[i])&(id_child_next==id_child_current[i])&(id_generation_next<30)) #Also add id_generations<30
    print("\nFound the id",id_next[match],"at the index",match[0],"in this snapshot")
    ind=np.append(ind,match)
    ind_tracked_id_next=ind[1:len(ind)]  #The extra element in the beginning is removed by this process
    print("\nThese are the indices of the ids that matched in current snapshot\n",ind_tracked_id_next)
    return ind_tracked_id_next  
################################################################
################################################################

In [9]:
total_clusters=len(cluster_dic)         #count the total no. of clusters we have in clusters file
total_snaps=len(snaps)  #count the total no. of snapshots we are going to track
snap=snaps[0]                      #Mark the beginning of the snapshot
test_cluster=1                             #Mark the beginning of the cluster that we would be tracking
ind_tracked={}                             #finding the indices of the tracked stars in each snapshot for each cluster.

tracked_data_all_clusters_each_snap={}     #dictionary for storing data from all clusters and in each snapshot

In [14]:
snaps = [412]

for snap in snaps:               #we run a for loop until the end of all snapshots
    part=gizmo.io.Read.read_snapshots(['star'],'index', snap, simulation_name=simname, simulation_directory=simdir, assign_hosts=True, assign_hosts_rotation=True)               #snap is the snapshot number here that changes everytime the loop iterates. It starts with sanpshot_start

    id=part['star'].prop('id')
    id_child=part['star'].prop('id.child')
    id_generation=part['star'].prop('id.generation')
    age=part['star'].prop('age')
    x=part['star'].prop('host.distance.principal')[:,0] #x component of the position of all stars 
    y=part['star'].prop('host.distance.principal')[:,1] #y component of the position of all stars
    z=part['star'].prop('host.distance.principal')[:,2] #z component of the position of all stars
    n=len(x) # counting the total no. of star particles
    vx=part['star'].prop('host.velocity.principal')[:,0]
    vy=part['star'].prop('host.velocity.principal')[:,1]
    vz=part['star'].prop('host.velocity.principal')[:,2]
    mass=part['star']['mass'] #mass of all stars in snapshot 691
    
#     above is simulation data without transformation

#  transformation code 
# save output into new set of variables = data with transformation

    
    
    
    #compute 3-D velocity in cylindrical coordinates
    # first value is along the major axes (positive definite)
    # secod value is azimuthal velocity in the plane of the disk (positive definite)
    # third value is vertical velocity wrt the disk (signed)
    vR_cyl=part['star'].prop('host.velocity.principal.cylindrical')[:,0]
    vphi_cyl=part['star'].prop('host.velocity.principal.cylindrical')[:,1]
    vz_cyl=part['star'].prop('host.velocity.principal.cylindrical')[:,2]
    print("\n#########\n############\nLoaded id,id_child,age,x,y,z,vx,vy,vz,mass, cylindrical velocities and number of particles for snapshot no.",snap)
    print("Total no of particles in this shapshot no.",snap,"is",n)
    
 ### Now matching the IDs of this snapshot with the ids of all clusters


    test_cluster=1                       #This is the test cluster to begin with. It resets to 1 at each snapshot 
    tracked_data_all_clusters={}         #This dictionary keeps the tracked information for all star clusters in given snapshot. It also resets with snapshots.
    for j in range(total_clusters):
        tracked_data={}                    #This dictionary holds the tracked information for a cluster temporarily until it is pushed to tracked_data_all_clusters
        print("\n\nNow Matching the ids of the snapshot",snap," with the cluster group id",test_cluster)
        id_test_cluster=cluster_dic[test_cluster]["id"]
        id_child_test_cluster=cluster_dic[test_cluster]["id_children"]
        sortind=np.argsort(id_test_cluster)
        id_test_cluster_sorted=id_test_cluster[sortind]
        id_child_test_cluster_sorted=id_child_test_cluster[sortind]
        print("The total no. of stars in cluster id ",test_cluster," is",len(id_test_cluster_sorted))
        print("Sorted ids of this cluster is",id_test_cluster_sorted)
        ind_tracked=matchids(id_test_cluster_sorted,id_child_test_cluster_sorted,id,id_child,id_generation)
    
    ################################################################
    ################################################################
    #Now finding the postion, velocities, age and mass of the tracked stars in each snapshots
        age_tracked=age[ind_tracked]
        x_tracked=x[ind_tracked]
        y_tracked=y[ind_tracked]
        z_tracked=z[ind_tracked]
        vx_tracked=vx[ind_tracked]
        vy_tracked=vy[ind_tracked]
        vz_tracked=vz[ind_tracked]
        mass_tracked=mass[ind_tracked]
        
        vR_cyl_tracked=vR_cyl[ind_tracked]
        vphi_cyl_tracked=vphi_cyl[ind_tracked]
        vz_cyl_tracked=vz_cyl[ind_tracked]
    
    
    ################################################################
    ################################################################
        #Now calculating center of mass and related properties  
        print(mass_tracked)
        xcm=distance_functions.cm(x_tracked,mass_tracked)
        ycm=distance_functions.cm(y_tracked,mass_tracked)
        zcm=distance_functions.cm(z_tracked,mass_tracked)
        delta_rxyz=distance_functions.dr(x_tracked,y_tracked,z_tracked,mass_tracked)
        rmax=distance_functions.drmax(x_tracked,y_tracked,z_tracked,mass_tracked)
        ymax=(ycm+1.1*rmax)
        ymin=(ycm-1.1*rmax)
        xmax=(xcm+1.1*rmax)
        xmin=(xcm-1.1*rmax)
        avg_delta_rxyz=np.mean(np.absolute(delta_rxyz))
  
  
    ################################################################
    ################################################################
    #Now lets write our tracked data from each snapshots to a file
        tracked_data={"ind_tracked":ind_tracked,"age_tracked":age_tracked,"x_tracked":x_tracked,"y_tracked":y_tracked,"z_tracked":z_tracked,
        "vx_tracked":vx_tracked,"vy_tracked":vy_tracked,"vz_tracked":vz_tracked,
        "mass_tracked":mass_tracked,"xcm":xcm,"ycm":ycm,"zcm":zcm,"delta_rxyz":delta_rxyz,"rmax":rmax,
        "ymax":ymax,"ymin":ymin,"xmax":xmax,"xmin":xmin,"avg_delta_rxyz":avg_delta_rxyz,
        "vR_cyl_tracked":vR_cyl_tracked,"vphi_cyl_tracked":vphi_cyl_tracked,"vz_cyl_tracked":vz_cyl_tracked}

    
        tracked_data_all_clusters.update({test_cluster:tracked_data}) #access it using tracked_data_all_clusters[clusterid]["key"]
        test_cluster+=1
  
  #We have collected all tracked information for our test clusters for one snapshot now which is in dictionary tracked_data_all_clusters.
  #We would store that dictionary in a pickle file as a backup data from each snapshot about or all star cluster.
  ###Now moving out of the j loop. We scanned all clusters and stored the tracked data into a dictionary tracked_data_all_clusters  
    file_name="tracked_clusters_at_snapshot_"+str(snap)+".pkl"
    with open(path+file_name, 'wb') as output:
        pickle.dump(tracked_data_all_clusters, output)
    print("\n Stored tracked data of all stars clusters in the snapshot no.",snap,"to filename:",file_name,"\n#####\n")
  #Note we are still in i loop which is scanning each snapshot !!!!!!!!!!!!!!!!!!
    tracked_data_all_clusters_each_snap.update({snap:tracked_data_all_clusters}) 

 
#Came out of i loop 
  
#Finally we scanned through all the snapshots and now we come out of the loop to store the final file that contains all the tracked information of all clusters 
#Now we store the tracked data of all clusters into a dictionary with snapshot number as the key. This would be our final dictionary of dictinaries. 
with open(path+"total_data_tracked_clusters_all_snapshots.pkl", 'wb') as output:
    pickle.dump(tracked_data_all_clusters_each_snap, output) #access the data using tracked_data_all_clusters_each_snap[snapshot][cluster_id]["x_tracked"])
  
#Expected Result: tracked_data_all_clusters_each_snap={596:{1:tracked_data,2:tracked_data..},597:{1:tracked_data,2:tracked_data..upto total clusters},598....     upto total snapshots}


# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/output/snapshot_times.txt

* reading:  scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/output/snapshot_scalefactors.txt

* reading:  scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/output/snapshot_scale-factors.txt

cannot find file of snapshot times in /scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/output/

# in utilities.io.get_file_names():
! cannot find files with base name:  /scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/output/output/snap*[!.txt!.ewah]


OSError: cannot find snapshot index = 412 in:  []

In [ ]:
cluster_dic.keys()
tracked_data_all_clusters_each_snap.keys()

In [ ]:
# print out masses see when it is giving me zero

In [ ]:
path_2="/Network/Servers/VALIVARRE.PHY.TCU.EDU/Volumes/SDF2/aibrahim/Research_with_Sarah/Plots/"
# if not os.path.exists(path):
#   os.makedirs(path)


tracked_file ="total_data_tracked_clusters_all_snapshots.pkl"
# temporary_name = simname+'.'+str(snaps[0])+".minage"+ str('%.2f' %min_age_array_Gyr[k])+"_"+"maxage"+"_"+str('%.2f' %max_age_array_Gyr[k])+"_b"+str(b_parsec)+"_nmin"+str(ncut_min)+".pkl"
with open(path_2+tracked_file, "rb") as fp:
    tracked_data_all_clusters_each_snap = pickle.load(fp)

change aspect ratio CHECK
plot rcm vs. zcm
legend by snapshot number

In [ ]:
snaps = [412,446,486,534,590,591,592,593,594,595,596,597,598,599,600]

fig2 = plt.figure()
ax2 = plt.gca()

for m in range(2,3):
    for i in snaps:
        
        x_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["xcm"]
        y_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["ycm"]
        z_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["zcm"]
        rcm = np.sqrt(x_tracked_cluster**2+y_tracked_cluster**2)
        ax2.scatter(rcm,z_tracked_cluster,label=i)
        ax2.set_xlabel("X [kpc]")
        ax2.set_ylabel("Z [kpc]")
#         ax2.set_aspect('equal')
#         ax2.legend()
    plt.show()


In [ ]:

rcm_values = []
zcm_values = []
for m in range(1,2):
    for i in snaps:
        snaps = [412,446,486,534,590,591,592,593,594,595,596,597,598,599,600]

        x_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["xcm"]
        y_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["ycm"]
        z_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["zcm"]
        rcm = np.sqrt(x_tracked_cluster**2+y_tracked_cluster**2)
        rcm_values.append(rcm)
        zcm_values.append(z_tracked_cluster)
#         ax2.scatter(np.array(rcm),np.array(z_tracked_cluster),c=snaps,cmap='Purples')
#         ax2.set_xlabel("rcm [kpc]")
#         ax2.set_ylabel("Z [kpc]")
#         ax2.set_aspect('equal')
# #         print(x_tracked_cluster)
#         print(rcm)
#         print(z_tracked_cluster)

        #         ax2.legend()
#     plt.show()


In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.scatter(rcm_values,zcm_values,c=snaps,cmap='gist_rainbow')
ax.text(3.78,-0.52,'snap 600')
ax.text(3.27,-0.27,'Snap 446')
ax.text(5.8,1.47,'Snap 486')
ax.text(3.58, 1.4372971018024545,'Snap 534')
ax.text(3.99,-0.3500246096634756,'Snap 599')
ax.set_xlabel('Rcm [Kpc]')
ax.set_ylabel('Zcm [Kpc]')
ax.set_aspect('equal')


In [ ]:
# ###### snaps = [446,486,534,590,591,592,593,594,595,596,597,598,599,600]

# fig2 = plt.figure()
# ax2 = plt.gca()

rcm_values = []
zcm_values = []
for m in range(2,3):
    for i in snaps:
        snaps = [412,446,486,534,590,591,592,593,594,595,596,597,598,599,600]

        x_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["xcm"]
        y_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["ycm"]
        z_tracked_cluster = tracked_data_all_clusters_each_snap[i][m]["zcm"]
        rcm = np.sqrt(x_tracked_cluster**2+y_tracked_cluster**2)
        rcm_values.append(rcm)
        zcm_values.append(z_tracked_cluster)
#         ax2.scatter(np.array(rcm),np.array(z_tracked_cluster),c=snaps,cmap='Purples')
#         ax2.set_xlabel("rcm [kpc]")
#         ax2.set_ylabel("Z [kpc]")
#         ax2.set_aspect('equal')
# #         print(x_tracked_cluster)
#         print(rcm)
#         print(z_tracked_cluster)

#         #         ax2.legend()
#     plt.show()


In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.scatter(rcm_values,zcm_values,c=snaps,cmap ='gist_rainbow')
ax.text(4.3,-0.93,'snap 600')
ax.text(3.77 ,-1.104,'Snap 446')
ax.text(4.76,-0.74,'Snap 486')
ax.text(5.27,-0.93,'Snap 534')
ax.text(4.46,-1.12,'Snap 599')
ax.set_xlabel('Rcm [Kpc]')
ax.set_ylabel('Zcm [Kpc]')
ax.set_aspect('equal')
